## MATH 4/5388: Machine Learning Methods 


### Module 2: Parametric Regression Models 

#### Instructor: Farhad Pourkamali 

<img src="CUDenverLogo.png" width=500 height=400   align="right">


#### Spring 2023

Textbook: Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow (Chapter 4)







### Overview of Module 2
<hr style="border:2px solid gray">

* Linear regression: Problem formulation, assumption, loss function, gradient
    * Normal equation
    * Sklearn implementation
    * Evaluation metrics
    * Gradient descent (GD) and variants
    * Nonlinear extension and regularization 
    * Model selection: The process of selecting the proper level of flexibility 

### Case study: univariate linear regression

<hr style="border:2px solid gray">
   
* Training data: $\mathcal{D}=\{(x_n,y_n)\}_{n=1}^N$

* Parametric model: $f(x)=\theta_0+\theta_1 x$

* Objective: Choose $\theta_0,\theta_1$ such that $f(x_n)$ is close to $y_n$ 

* Mean squared error (MSE): 

$$\mathcal{L}(\theta_0, \theta_1)=\frac{1}{N}\sum_{n=1}^N \big(y_n - f(x_n)\big)^2$$


<center><img src="uni.png" width=800 height=700><center>




### Solving the optimization problem

<hr style="border:2px solid gray">

* We'll need the concept of partial derivatives

* To compute $\partial \mathcal{L}/\partial \theta_0$, take the  derivative with respect to $\theta_0$, treating the rest of the arguments as constants

* We can show that 

$$\frac{\partial \mathcal{L}}{\partial \theta_0}=\frac{-2}{N}\sum_{n=1}^N\big(y_n-\theta_0-\theta_1x_n\big)=\frac{2}{N}\sum_{n=1}^N\big(f(x_n)- y_n\big)$$

$$\frac{\partial \mathcal{L}}{\partial \theta_1}=\frac{-2}{N}\sum_{n=1}^N\big(y_n-\theta_0-\theta_1x_n\big)x_n=\frac{2}{N}\sum_{n=1}^N\big(f(x_n)- y_n\big)x_n$$

### Gradient 
<hr style="border:2px solid gray">


* Extend the notion of derivatives to handle vector-argument functions
    + Given $\mathcal{L}:\mathbb{R}^d\mapsto \mathbb{R}$, where $d$ is the number of input variables
    
    $$\nabla \mathcal{L}=\begin{bmatrix}\frac{\partial \mathcal{L}}{\partial \theta_0}\\ \vdots\\ \frac{\partial \mathcal{L}}{\partial \theta_{d-1}} \end{bmatrix}\in\mathbb{R}^d$$
    
* Example from the previous slide ($d=2$): 

$$\nabla \mathcal{L}=\frac{2}{N}\begin{bmatrix}\sum_{n=1}^N\big(f(x_n) - y_n\big)\\ \sum_{n=1}^N\big(f(x_n) - y_n\big)x_n \end{bmatrix}\in\mathbb{R}^2$$

### Compact form of gradient 
<hr style="border:2px solid gray">

* Let us define  

$$\mathbf{X}=\begin{bmatrix} 1 & x_1\\ \vdots & \vdots \\ 1 & x_N \end{bmatrix}\in\mathbb{R}^{N\times 2}, \boldsymbol{\theta}=\begin{bmatrix}\theta_0 \\ \theta_1\end{bmatrix}\in\mathbb{R}^{2}, \mathbf{y}=\begin{bmatrix}y_1 \\ \vdots \\ y_N\end{bmatrix}\in\mathbb{R}^{N}$$

* Hence, we get 

$$\mathbf{X}\boldsymbol{\theta} - \mathbf{y}=\begin{bmatrix}f(x_1) - y_1 \\ \vdots \\ f(x_N) - y_N\end{bmatrix}$$


### Compact form of gradient 
<hr style="border:2px solid gray">

* The last step is to show that $\nabla \mathcal{L}$ can be written as 

$$\frac{2}{N}\mathbf{X}^T\big(\mathbf{X}\boldsymbol{\theta} - \mathbf{y}\big)=\frac{2}{N}\begin{bmatrix} 1 & \ldots & 1\\x_1 & \ldots & x_N\end{bmatrix}\begin{bmatrix}f(x_1) - y_1 \\ \vdots \\ f(x_N) - y_N\end{bmatrix}=\frac{2}{N}\begin{bmatrix}\sum_{n=1}^N\big(f(x_n) - y_n\big)\\ \sum_{n=1}^N\big(f(x_n) - y_n\big)x_n \end{bmatrix}$$

* Given this compact form, we can use NumPy to solve the linear matrix equation

$$\underbrace{\mathbf{X}^T\mathbf{X}}_{a}\boldsymbol{\theta}=\underbrace{\mathbf{X}^T\mathbf{y}}_{b}$$

<center><img src="lstsq.png" width=700 height=600><center>



In [ ]:
# GDP data 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

df = pd.read_csv("https://github.com/ageron/data/raw/main/lifesat/lifesat.csv")
                 
df.head()

In [ ]:
X = df['GDP per capita (USD)'].to_numpy()
y = df['Life satisfaction'].to_numpy()

print(X.shape, y.shape)

In [ ]:
# add the column of all 1's 

def add_column(X):
    '''
    add the column of all 1's 
    '''
    return np.concatenate(( np.ones((X.shape[0],1)), X.reshape(-1,1)), axis=1)

Xcon = add_column(X)

Xcon.shape

In [ ]:
# solve the problem 

a = np.matmul(np.transpose(Xcon), Xcon)

b = np.matmul(np.transpose(Xcon), y)

theta = np.linalg.lstsq(a, b, rcond=None)[0] # Cut-off ratio for small singular values

print(theta)

In [ ]:
# plot the prediction model f

def f(X, theta):
    return np.matmul(X, theta)

plt.rcParams.update({'font.size': 12, "figure.figsize": (6,3)})
plt.scatter(X, y, s=20, label='training data')

X_test = np.array([25000, 60000])
y_test = f(add_column(X_test), theta)
plt.plot(X_test, y_test, 'r--', label='prediction')

plt.legend()
plt.xlabel('GDP')
plt.ylabel('life satisfaction')
plt.show()


### Linear models for regression
<hr style="border:2px solid gray">

* Given the training data set $\mathcal{D}=\{(\mathbf{x}_n,y_n)\}_{n=1}^N$ and an input vector $\mathbf{x}\in\mathbb{R}^D$, the linear regression model takes the form

$$f(\mathbf{x})=\theta_0+\theta_1x_1+\theta_2x_2+\ldots+\theta_Dx_D=\theta_0+\boldsymbol{\theta}^T\mathbf{x}$$
    
* $\boldsymbol{\theta}\in\mathbb{R}^D$: weights or regression coefficients, $\theta_0$: intercept or bias term

* Compact representation by defining $\mathbf{x}=[\color{red}{x_0=1},x_1,\ldots,x_D]$ and $\boldsymbol{\theta}=[\theta_0,\theta_1,\ldots,\theta_D]$ in $\mathbb{R}^{D+1}$

$$f(\mathbf{x})=\boldsymbol{\theta}^T\mathbf{x}=\langle \boldsymbol{\theta}, \mathbf{x}\rangle$$

### Loss function for linear regression
<hr style="border:2px solid gray">

* MSE loss function for a linear regression model

$$\mathcal{L}(\boldsymbol{\theta})=\frac{1}{N}\sum_{n=1}^N\big(y_n-\langle \boldsymbol{\theta},\mathbf{x}_n\rangle\big)^2=\frac{1}{N}\|\mathbf{y}-\mathbf{X}\boldsymbol{\theta}\|_2^2$$

where we have

$$\mathbf{X}=\begin{bmatrix}\rule[.5ex]{1em}{0.4pt}\mathbf{x}_1^T \rule[.5ex]{1em}{0.4pt}\\ \vdots\\\rule[.5ex]{1em}{0.4pt}\mathbf{x}_N^T \rule[.5ex]{1em}{0.4pt}\end{bmatrix}\in\mathbb{R}^{N\times (D+1)}, \boldsymbol{\theta}=\begin{bmatrix}
\theta_0\\\theta_1\\ \vdots \\\theta_{D}\end{bmatrix}\in\mathbb{R}^{D+1},\;\mathbf{y}=\begin{bmatrix}y_1\\y_2\\ \vdots\\y_N\end{bmatrix}\in\mathbb{R}^N, $$

* Optimization problem for model fitting/training: $\underset{\boldsymbol{\theta}\in\mathbb{R}^{D+1}}{\operatorname{argmin}} \mathcal{L}(\boldsymbol{\theta})$




### The Normal equation
<hr style="border:2px solid gray">

* To find the value of $\boldsymbol{\theta}$ that minimizes the MSE, there exists a *closed-form* solution
    * a mathematical equation that gives the result directly
    
* The gradient takes the form 

$$\nabla \mathcal{L}(\boldsymbol{\theta})= \frac{2}{N}\mathbf{X}^T\big(\mathbf{X}\boldsymbol{\theta} - \mathbf{y}\big)$$

* Normal equation

$$\boldsymbol{\theta}^*=\big(\mathbf{X}^T\mathbf{X}\big)^{-1}\mathbf{X}^T\mathbf{y}$$

### Sklearn implementation
<hr style="border:2px solid gray">

* Documentation page: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html 

    * Parameters: Useful for creating objects 
    
    * Attributes: Estimated coefficients, etc.
    
    * Methods: Training, prediction, etc.
    
<center><img src="linearreg.png" width=900 height=800><center>

In [ ]:
# Revisit the GDP data 

from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(X.reshape(-1,1), y) # X should be a 2D array 

print(reg.intercept_, reg.coef_)


### Preprocessing data
<hr style="border:2px solid gray">

* Change raw/original feature vectors into a representation that is more suitable for the downstream estimators/tasks

* The *sklearn.preprocessing* package provides several common utility functions
    * Scaling features to lie between a given minimum and maximum value
    
    * Removing the mean value and dividing features by their standard deviation
        * features look like standard normally distributed data
    
* We use the first technique in the next slide

In [ ]:
# Revisit the GDP data 

from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

X_minmax = minmax.fit_transform(X.reshape(-1,1))

reg = LinearRegression()

reg.fit(X_minmax, y) # X should be a 2D array 

print(reg.intercept_, reg.coef_)

In [ ]:
# Plot the prediction model 

plt.rcParams.update({'font.size': 12, "figure.figsize": (6,3)})
plt.scatter(X_minmax, y, s=20, label='training data')

X_test = np.array([25000, 60000]).reshape(-1,1)
X_test_minmax = minmax.transform(X_test)
plt.plot(X_test_minmax, reg.predict(X_test_minmax), 'r--', label='prediction')

plt.legend()
plt.xlabel('GDP (scaled)')
plt.ylabel('life satisfaction')
plt.show()

### Evaluation metrics for regression problems
<hr style="border:2px solid gray">

* The quality of a regression model can be assessed using various quantities
    * See https://scikit-learn.org/stable/modules/model_evaluation.html

* Mean squared error 
$$\text{MSE}(\mathbf{y}, \hat{\mathbf{y}})=\frac{1}{N_{\text{test}}}\sum_{n=1}^{N_{\text{test}}} \big(y_n - \hat{y}_n\big)^2$$

    * The value you get after calculating MSE is a squared unit of output
    
    * If you have outliers in the data set, then it penalizes the outliers most 

### Root Mean squared error (RMSE)
<hr style="border:2px solid gray">

* The output value you get is in the same unit as the required output variable

$$\text{RMSE}(\mathbf{y}, \hat{\mathbf{y}})=\sqrt{\frac{1}{N_{\text{test}}}\sum_{n=1}^{N_{\text{test}}} \big(y_n - \hat{y}_n\big)^2}$$

In [ ]:
from sklearn.metrics import mean_squared_error
y_true = [3, -1, 2, 7]
y_pred = [3, 0, 2, 7]

# If True returns MSE value, if False returns RMSE value.
print('MSE: ', mean_squared_error(y_true, y_pred), 
      ', RMSE: ', mean_squared_error(y_true, y_pred, squared=False))

### R² score or the coefficient of determination
<hr style="border:2px solid gray">

* Definition 

$$R^2(\mathbf{y}, \hat{\mathbf{y}}) = 1 - \frac{\sum_{n=1}^{N_{\text{test}}} (y_n - \hat{y}_n)^2}{\sum_{n=1}^{N_{\text{test}}} (y_n - \bar{y})^2},\;\;\bar{y} = \frac{1}{N_{\text{test}}} \sum_{n=1}^{N_{\text{test}}} y_n$$

* RSS (Residual Sum of Squares) measures the amount of variability that is left unexplained 

$$\text{RSS}=\sum_{n=1}^{N_{\text{test}}} (y_n - \hat{y}_n)^2$$

* Best possible score is 1 and a number near 0 indicates the model does not explain much of the variability in the response

### Explained variance score
<hr style="border:2px solid gray">

* Definition

$$explained\_{}variance(\mathbf{y}, \hat{\mathbf{y}}) = 1 - \frac{Var\{ \mathbf{y} - \hat{\mathbf{y}}\}}{Var\{\mathbf{y}\}}$$

* The best possible score is 1.0, lower values are worse

* When the prediction residuals have zero mean, the $R^2$ score and the Explained variance score are identical


In [ ]:
from sklearn.metrics import r2_score, explained_variance_score
y_true = [3, -1, 2, 7]
y_pred = [2.9, 0, 2.5, 6.5]
r2_score(y_true, y_pred), explained_variance_score(y_true, y_pred)

### Gradient descent (GD)
<hr style="border:2px solid gray">

* Tweak parameters $\boldsymbol{\theta}$ iteratively to minimize the loss function
$\mathcal{L}(\boldsymbol{\theta})$


* At each iteration $t$, perform an update to decrease the loss function

$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t - \eta_t \nabla \mathcal{L}(\boldsymbol{\theta}_t)$$

where $\eta_t$ is the step size or learning rate

    
<center><img src="gd.png" width=600 height=400 ><center>

### Learning rate hyperparameter
<hr style="border:2px solid gray">

* A hyperparameter is a parameter that is set *before* the learning process begins
    * These parameters are tunable and directly affect how well a model trains
    
* If the learning rate is too small, then the algorithm will have to go through many iterations to converge
* The algorithm may diverge when the learning rate is too high 

<center><img src="learning_rate.png" width=800 height=600 ><center>
    

### One-Dimensional Gradient Descent
<hr style="border:2px solid gray">

* Consider some continuously differentiable real-valued function $f: \mathbb{R} \rightarrow \mathbb{R}$.  Using a Taylor expansion we obtain

$$f(x + \epsilon) = f(x) + \epsilon f'(x) + \mathcal{O}(\epsilon^2)$$

* Choose $\epsilon = -\eta f'(x)$ for $\eta>0$, we get

$$f(x - \eta f'(x)) = f(x) - \eta f'^2(x) + \mathcal{O}(\eta^2 f'^2(x))$$

* choose $\eta$ small enough for the higher-order terms to become irrelevant

$$f(x - \eta f'(x)) \lessapprox f(x)$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):  # Objective function
    return x ** 2

def f_grad(x):  # Gradient (derivative) of the objective function
    return 2 * x

def gd(eta, f_grad):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * f_grad(x)
        results.append(float(x))
    return results

In [ ]:
def show_trace(results, f):
    
    n = max(abs(min(results)), abs(max(results)))
    f_line = np.arange(-n, n, 0.1)
    plt.plot(f_line, [f(x) for x in f_line], 'r-')
    
    for i in range(0, len(results), 1):
        plt.plot(results[i:i+2], [f(x) for x in results[i:i+2]], 'bo-')
    
    
    plt.show()

In [ ]:
show_trace(gd(1, f_grad), f)

### Batch GD for linear regression
<hr style="border:2px solid gray">

* Recall the gradient vector of the loss function

$$\nabla \mathcal{L}(\boldsymbol{\theta})= \frac{2}{N}\mathbf{X}^T\big(\mathbf{X}\boldsymbol{\theta} - \mathbf{y}\big)$$

* GD step with fixed learning rate
$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t - \eta \nabla \mathcal{L}(\boldsymbol{\theta}_t)=\boldsymbol{\theta}_t - \eta \frac{2}{N}\mathbf{X}^T\big(\mathbf{X}\boldsymbol{\theta}_t - \mathbf{y}\big) $$

    * This formula involves calculations over the full training set $\mathbf{X}$ --> batch or full GD
    * An epoch means one complete pass of the training data set 



In [ ]:
# GDP data 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

df = pd.read_csv("https://github.com/ageron/data/raw/main/lifesat/lifesat.csv")

X = df['GDP per capita (USD)'].to_numpy()
y = df['Life satisfaction'].to_numpy()

def add_column(X):
    '''
    add the column of all 1's 
    '''
    return np.concatenate(( np.ones((X.shape[0],1)), X.reshape(-1,1)), axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

X_minmax = minmax.fit_transform(X.reshape(-1,1))

Xcon = add_column(X_minmax)

print(Xcon[:3])

In [ ]:
# Implementation of Batch GD 

eta = 0.01  # learning rate
n_epochs = 1000
N = len(Xcon)  # number of instances

np.random.seed(3)
theta = np.random.randn(2, 1)  # randomly initialized model parameters

for epoch in range(n_epochs):
    gradients = 2 / N * Xcon.T @ (Xcon @ theta - y.reshape(-1,1))
    theta = theta - eta * gradients
    
print(theta)

In [ ]:
# impact of epoch and eta 
np.random.seed(3)
theta = np.random.randn(2, 1)  # randomly initialized model parameters

X_test = np.array([25000, 60000]).reshape(-1,1)
X_test_minmax = add_column(minmax.transform(X_test))
plt.scatter(X_minmax, y, s=20, label='training data')
eta= .1 # 0.001, 0.01, 0.1, 1
for epoch in range(n_epochs):
    gradients = 2 / N * Xcon.T @ (Xcon @ theta - y.reshape(-1,1))
    theta = theta - eta * gradients
    if epoch == 1: 
        plt.plot(X_test_minmax[:,1], X_test_minmax@theta , 'b:', label='1 epochs')
    elif epoch == 10:
        plt.plot(X_test_minmax[:,1], X_test_minmax@theta , 'r--', label='10 epochs')
    elif epoch == 100:
        plt.plot(X_test_minmax[:,1], X_test_minmax@theta , 'g-', label='100 epochs')           
plt.legend()
plt.xlabel('GDP (scaled)')
plt.ylabel('life satisfaction')
plt.show()

### Stochastic gradient descent (SGD) for linear regression
<hr style="border:2px solid gray">

* The main problem with GD is that it uses the whole training set at every step 
* Consider a minibatch of size $B=1$ and a selected sample $\mathbf{x}_n^T$ from $\mathbf{X}$ (row vector)

$$\nabla \mathcal{L}(\boldsymbol{\theta})=\frac{2}{N}\mathbf{X}^T(\mathbf{X}\boldsymbol{\mathbf{w}}-\mathbf{y}) \Rightarrow 2\mathbf{x}_n(\mathbf{x}_n^T\boldsymbol{\theta}-y_n)$$

$$\boldsymbol{\theta}_{t+1}=\boldsymbol{\theta}_t - 2\mathbf{x}_n(\mathbf{x}_n^T\boldsymbol{\theta}-y_n)$$

* Given that $N$ is the sample size and $B$ is the batch size, in one epoch we update our model $N/B$ times
  

In [ ]:
n_epochs = 5
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

np.random.seed(42)
theta = np.random.randn(2, 1)  # random initialization

for epoch in range(n_epochs):
    for iteration in range(N):
        random_index = np.random.randint(N)
        xi = np.transpose(Xcon[random_index : random_index + 1])
        yi = y[random_index : random_index + 1]
        gradients = 2 * xi @ (xi.T @ theta - yi)  # for SGD, do not divide by N
        eta = learning_schedule(epoch * N + iteration)
        theta = theta - eta * gradients
        
print(theta)

### Sklearn implementation of SGD for linear regression
<hr style="border:2px solid gray">

* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

* Parameters
    * max_iter: epochs
    * learning_rate: constant or variable
    * n_iter_no_change: number of iterations with no improvement to wait before stopping fitting
    
<center><img src="SGDRegressor.png" width=800 height=600 ><center>

In [ ]:
# GDP data 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor

X = df['GDP per capita (USD)'].to_numpy().reshape(-1,1)
y = df['Life satisfaction'].to_numpy()

pipe = Pipeline([('preprocess', MinMaxScaler()), 
                 ('reg', SGDRegressor(random_state=42))])

pipe.fit(X, y)

print(pipe['reg'].intercept_, pipe['reg'].coef_)


### Polynomial regression 
<hr style="border:2px solid gray">

* The linear model may not be a good fit for many problems 
    * We can improve the fit by using a polynomial regression model of degree $d$

$$f(x)=\boldsymbol{\theta}^T\phi(x)$$

where $\phi(x)=[1,x,x^2,\ldots,x^d]$

* This is a simple example of feature preprocessing/engineering
    * Benefit: linear function of parameters but nonlinear wrt input features

* We can use [sklearn.preprocessing.PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to generate polynomial features
    * Use pipeline in sklearn to assemble several steps (preprocessing + estimator)



In [ ]:
# generate simulated data 
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
plt.rcParams.update({'font.size': 16, "figure.figsize": (6,4)})

N = 400
X = 6 * np.random.rand(N, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(N, 1)

plt.plot(X, y, "b.")
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()

In [ ]:
# train polynomial model 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
pipe = Pipeline(steps=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('regr', LinearRegression())])
pipe.fit(X, y) # training 
X_new = np.linspace(-3, 3, 100).reshape(100, 1)
y_new = pipe.predict(X_new) # prediction

plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([-3, 3, 0, 10])
plt.show()

In [ ]:
# Compare varying complexity levels 
from sklearn.preprocessing import StandardScaler
for style, width, degree in (("g-", 2, 30), ("b--", 1, 2), ("r-+", 1, 1)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = LinearRegression()
    polynomial_regression = Pipeline([("poly_features", polybig_features),
                                      ("std_scaler", std_scaler),
                                      ("lin_reg", lin_reg)])
    polynomial_regression.fit(X, y)
    y_newbig = polynomial_regression.predict(X_new)
    plt.plot(X_new, y_newbig, style, label=str(degree), linewidth=width)
plt.plot(X, y, "b.", linewidth=3)
plt.legend()
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, -10, 10])
plt.show()

### The bias-variance tradeoff
<hr style="border:2px solid gray">

* A model’s generalization/test error can be expressed as the sum of 
    * Bias due to wrong assumptions
        * A high-bias model is likely to underfit the data
    
    * Variance due to excessive sensitivity to small variations in the training data
        * A high-variance model is likely to overfit the data
    
    * Irreducible error due to noisiness of the data itself

### The bias-variance tradeoff
<hr style="border:2px solid gray">

* As model flexibility increases, training error decreases, but there is a U-shape in test error

* In practice, computing training error is straightforward, but estimating test error is considerably more difficult because no test data are available

<center><img src="biasvar.png" width=700 height=600 ><center>

### Regularization
<hr style="border:2px solid gray">


* Regularization is a way to avoid overfitting by shrinking or simplifying the model  
    
    $$\mathcal{L}(\boldsymbol{\theta})=\|\mathbf{y}-\mathbf{X}\boldsymbol{\theta}\|_2^2+\lambda C(\boldsymbol{\theta})$$
    
    
* $\lambda\geq 0$ is the regularization parameter (i.e., hyperparameter) and $C(\boldsymbol{\theta})$ is some form of model complexity
* We can quantify complexity using the $\ell_2$ regularization formula, i.e., the sum of the squares of all weights

$$\ell_2 \text{ regularization}=\|\boldsymbol{\theta}\|_2^2=\theta_0^2+\theta_1^2+\theta_2^2+\ldots$$


### $\ell_1$ regularization or LASSO

<hr style="border:2px solid gray">

* LASSO: Least Absolute Shrinkage and Selection Operator 
    * Uses the $\ell_1$ norm of weights, instead of $\ell_2$
    
    $$\|\boldsymbol{\theta}\|_1=|\theta_0|+|\theta_1|+|\theta_2|+\ldots$$
          
* Definition of the $\ell_p$ norm for a real number $p\geq 1$

$$\|\boldsymbol{\theta}\|_p=\big(\sum_i |\theta_i|^p\big)^{1/p}$$
* LASSO tends to eliminate the weights of the least important features (i.e., set them to zero)
    * Next slide provides an intuitive explanation

### $\ell_1$ vs $\ell_2$ regularization 
<hr style="border:2px solid gray">

* Write optimization problems in bound constrained forms
$$\underset{\boldsymbol{\theta}}{\operatorname{min}} \mathcal{L}(\boldsymbol{\theta}) \;\text{ s.t. }\; \|\boldsymbol{\theta}\|_1\leq B \;\text{ or }\; \|\boldsymbol{\theta}\|_2^2\leq B$$

* Let us look at the contours of the $\ell_1$ and $\ell_2$ constrained surfaces
    * Corners of the $\ell_1$ ball are more likely to intersect the ellipse than one of the sides

<center><img src="opt.png" width=600 height=500 ><center>

In [ ]:
import numpy as np 

D = 5

np.random.seed(1)

theta = np.random.uniform(size=D)

theta /= np.linalg.norm(theta)

print(theta, np.linalg.norm(theta))


In [ ]:
num_exper = np.logspace(3,6,40)

result = np.zeros(len(num_exper))

sigma = 2

for ind in range(len(num_exper)):
    num = int(num_exper[ind])
    X = sigma * np.random.randn(num, D)
    result[ind] = np.mean(np.power(np.matmul(X, theta), 2))

plt.rcParams.update({'font.size': 12, "figure.figsize": (5,3)})
plt.plot(num_exper, result, 'C2-')
plt.axhline(sigma**2, c='C4')
plt.xscale('log')
plt.xlabel('number of experiments')
plt.ylabel('averaged value')
plt.show()

In [ ]:
# Synthetic/simulated data 
from sklearn.linear_model import Ridge
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

np.random.seed(42)
N = 20
X = 3 * np.random.rand(N, 1)
y = 1 + 0.5 * X + np.random.randn(N, 1) / 1.5
X_new = np.linspace(0, 3, 100).reshape(100, 1)
alphas=(0, 10**-5, 1000)
for alpha, style in zip(alphas, ("b-", "g--", "r:")): # zip: aggregates them in a tuple
    model = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("std_scaler", StandardScaler()),
        ("regul_reg", Ridge(alpha=alpha)), # alpha = lambda 
                    ])
    model.fit(X, y)
    plt.plot(X_new, model.predict(X_new), style, linewidth=2, label=r"$\alpha = {}$".format(alpha))
plt.plot(X, y, "k.", markersize=10)
plt.legend()
plt.show()

In [ ]:
# Lasso 
import numpy as np 
import pandas as pd 

# “Hitters” with 20 variables and 322 observations of major league players
df = pd.read_csv('Hitters.csv', index_col=[0])

# drop missing cases
df = df.dropna()

df.head()

In [ ]:
# Preprocessing, encode our categorical features as one-hot numeric features 

dummies = pd.get_dummies(df[['League', 'Division','NewLeague']])

dummies.head()

In [ ]:
# Find outputs 

y = df['Salary']

y.shape

In [ ]:
# drop the "Salary" column  and categorical columns 

df_input = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1)


# Create all features
X = pd.concat([df_input, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1).to_numpy()

X.shape

In [ ]:
# Split the data set into train and test set 70/30 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) # 15

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
X_train

### Should we use preprocessing?

* Let us try sklearn.preprocessing.StandardScaler

    * Standardize features by removing the mean and scaling to unit variance
    
    $$z_n=\frac{x_n - \mu}{\sigma}$$
    
    with mean 
    
    $$\mu=\frac{1}{N}\sum_{n=1}^N x_n$$
    
    and standard deviation 
    
    $$\sigma=\sqrt{\frac{1}{N}\sum_{n=1}^N (x_n - \mu)^2}$$

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

a = scaler.fit_transform(X_train[:,0:16])

b = scaler.transform(X_test[:,0:16])

X_train = np.concatenate((a, X_train[:,16:]), axis=1)

X_test = np.concatenate((b, X_test[:,16:]), axis=1)

X_train

In [ ]:
from sklearn.linear_model import Lasso

alphas = np.arange(10, 205, 10)

lasso = Lasso(max_iter=10000)

coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    coefs.append(lasso.coef_)

In [ ]:
coefs[0] # 16, 13

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 15, "figure.figsize": (14,5)})
max_vals = [item.max() for item in coefs]
min_vals = [item.min() for item in coefs]
zero_vals = [np.sum(item == 0) for item in coefs]
plt.subplot(121)
plt.plot(alphas, max_vals, 'b--')
plt.plot(alphas, min_vals, 'r--')
plt.xlabel('alpha')
plt.ylabel('max/min values')

plt.subplot(122)
plt.plot(alphas, zero_vals)
plt.xlabel('alpha')
plt.ylabel('number of zeros')
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

alphas = np.arange(10, 205, 10)

lasso = Lasso(max_iter=10000)

r2_values = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    r2_values.append(r2_score(y_test, lasso.predict(X_test)))
    

In [ ]:
plt.subplot(121)
plt.plot(alphas, r2_values, 'b--')
plt.plot(alphas[3], r2_values[3], 'ro')
plt.xlabel('alpha')
plt.ylabel('r2 values')
plt.grid()

plt.subplot(122)
plt.plot(alphas, zero_vals, 'b--')
plt.plot(alphas[3], zero_vals[3], 'ro')
plt.xlabel('alpha')
plt.ylabel('number of zeros')

plt.grid()
plt.show()

### Estimating generalization error
<hr style="border:2px solid gray">

* Two important tasks in machine learning 
    * having chosen a model, estimating its prediction error (generalization error) on new data -> model assessment
    * selecting the proper level of flexibility for a model -> model selection
    
* Question: how can we use the *available* data to perform model selection/assessment?

* We discuss two of the most commonly-used resampling methods
    * cross-validation 
    * bootstrap
    
*  Key concepts remain the same regardless of whether the response
is quantitative or qualitative

### Approach 1: Validation Set 
<hr style="border:2px solid gray">

* It involves *randomly* dividing the available set of observations into two parts, a training set and a validation set (or hold-out/test set)
 *  A model is fit on the training set, and its performance is evaluated on the validation set
 * If we repeat the process of randomly splitting the data, we will get different results (for example, the previous case study)
 * Only a subset of the data is used to fit the model -> tend to overestimate the generalization error

<center><img src="approach1.png" width=500 height=400><center>

### Approach 2:  Leave-One-Out Cross-Validation (LOOCV)
<hr style="border:2px solid gray">

* Use a single observation $(x_1,y_1)$ for the validation set and $\{(x_2,y_2),\ldots,(x_N,y_N)\}$ for model fitting to compute $\text{MSE}_1=(y_1-\hat{y}_1)^2$

* Repeat this process by selecting one sample for validation and the remaining $N-1$ for training 

$$\text{CV}=\frac{1}{N}\sum_{n=1}^N \text{MSE}_n$$

* There is no randomness in the training/validation set splits

<center><img src="approach2.png" width=400 height=300><center>

In [ ]:
import numpy as np
X, y = np.arange(10).reshape((5, 2)), np.arange(5)
print('X is: ', X, '\n', 'y is: ', y)

In [ ]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

### Approach 3:  K-Fold Cross-Validation
<hr style="border:2px solid gray">

* It involves randomly dividing the available data into $K$ groups, or folds, of approximately equal size
    * One fold is treated as a validation set and the remaining $K-1$ folds are used for training
    * Common choices $K=5$ or $10$
    * Advantage over LOOCV: computational savings
    
<center><img src="approach3.png" width=450 height=350><center>

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
from sklearn import datasets, linear_model

from sklearn.model_selection import cross_val_score

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target
lasso = linear_model.Lasso(alpha=0.01)
cv_results = cross_val_score(lasso, X, y, scoring='r2', cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" 
      % (cv_results.mean(), cv_results.std()))

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 12, "figure.figsize": (4,3)})

plt.boxplot(cv_results, vert=False)
plt.yticks([])
plt.xlabel('r2 values')
plt.show()

### Approach 4:  The Bootstrap
<hr style="border:2px solid gray">

* We obtain distinct data sets by repeatedly sampling observations from the original data set *with replacement*
    * Each of these “bootstrap data sets” is the same size as our original data
    
* A graphical illustration on a small data set with $N=3$

<center><img src="approach4.png" width=350 height=300><center>

### Model selection
<hr style="border:2px solid gray">

* Model selection is the process of selecting the best one by comparing and validating with various parameters
    * https://scikit-learn.org/stable/modules/cross_validation.html

<center><img src="selection.png" width=700 height=600 ><center>

In [ ]:
# Exhaustive search over specified parameter values for an estimator.

from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

parameters = {'alpha': np.linspace(1,300,100)}

lasso = Lasso(max_iter=10000)

reg = GridSearchCV(lasso, parameters, scoring='r2') 

reg.fit(X_train, y_train)

reg.best_params_

In [ ]:
# Set the best alpha

lasso_best = Lasso(alpha=reg.best_params_['alpha'], max_iter=10000)

lasso_best.fit(X_train, y_train)

lasso_best.coef_

In [ ]:
# R2 for training and test sets 

from sklearn.metrics import r2_score

pred_train = lasso_best.predict(X_train)
r2_train = r2_score(y_train, pred_train)
print('R2 training set', round(r2_train, 2))

# Test data
pred_test = lasso_best.predict(X_test)
r2_test = r2_score(y_test, pred_test)
print('R2 test set', round(r2_test, 2))

### Robust regression
<hr style="border:2px solid gray">

* When we have outliers, least squares doesn't always perform well!

* Thus, we have to modify least squares to pick a $\boldsymbol{\theta}$ that minimizes the general objective function

$$\sum_{n=1}^{N} g\big(y_n - \boldsymbol{\theta}^T\mathbf{x}_n\big)$$


* Instead of using $g(r)=r^2$, we can use the Huber's method

$$g(r)=\begin{cases}r^2/2 & \text{if } |r|<\delta \\
\delta|r| - \delta^2/2 & \text{otherwise}\end{cases}$$

<center><img src="huber.png" width=800 height=700 ><center>

In [ ]:
import numpy as np 
from scipy.special import huber
import matplotlib.pyplot as plt 

r = np.linspace(-6,6,100)

deltas = [.5, 1, 2]
linestyles = ["dashed", "dotted", "dashdot"]
fig, ax = plt.subplots()
combined_plot_parameters = list(zip(deltas, linestyles))
for delta, style in combined_plot_parameters:
    ax.plot(r, huber(delta, r), label=f"$\delta={delta}$", ls=style)
    
ax.plot(r, r**2, 'r-', label='quadratic')
ax.legend(loc="upper center")
ax.set_xlabel("$r$")
ax.set_title("Huber loss")
ax.set_xlim(-6, 6)
ax.set_ylim(0, 11)
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_regression
from sklearn.linear_model import HuberRegressor, Ridge

# Generate toy data
rng = np.random.RandomState(0)
X, y = make_regression(
    n_samples=20, n_features=1, random_state=0, noise=4.0, bias=100.0
)

plt.scatter(X, y)
plt.xlabel("X")
plt.ylabel("y")
plt.show()

In [ ]:
# Add four strong outliers to the dataset.
X_outliers = rng.normal(0, 0.5, size=(4, 1))
y_outliers = rng.normal(0, 2.0, size=4)

X_outliers[:2, :] += X.max() + X.mean() / 4.0
X_outliers[2:, :] += X.min() - X.mean() / 4.0
y_outliers[:2] += y.min() - y.mean() / 4.0
y_outliers[2:] += y.max() + y.mean() / 4.0

X = np.vstack((X, X_outliers))
y = np.concatenate((y, y_outliers))

plt.rcParams.update({'font.size': 12, "figure.figsize": (6,4)})
plt.scatter(X, y)
plt.xlabel("X")
plt.ylabel("y")
plt.show()

In [ ]:
# Fit the huber regressor over a series of epsilon values.
colors = ["r-", "b-", "y-", "m-"]

plt.rcParams.update({'font.size': 12, "figure.figsize": (6,4)})
x = np.linspace(X.min(), X.max(), 7)
epsilon_values = [1, 1.5, 1.75, 1.9] # like delta 
for k, epsilon in enumerate(epsilon_values):
    huber = HuberRegressor(alpha=0.0, epsilon=epsilon)
    huber.fit(X, y)
    coef_ = huber.coef_ * x + huber.intercept_
    plt.plot(x, coef_, colors[k], label="huber loss, %s" % epsilon)

plt.scatter(X, y)
plt.title("HuberRegressor")
plt.xlabel("X")
plt.ylabel("y")
plt.legend(loc=0)
plt.show()

### Multi target regression
<hr style="border:2px solid gray">

* We can use sklearn.multioutput.MultiOutputRegressor

    * This strategy consists of fitting one regressor per target

In [ ]:
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

X, y = load_linnerud(return_X_y=True)

print(X.shape, y.shape)

In [ ]:
# train/test split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 

# train 
regr = MultiOutputRegressor(Ridge()).fit(X_train, y_train)

# predict 

regr.predict(X_test)

In [ ]:
regr.estimators_

In [ ]:
for i in range(3): 
    print(np.dot(regr.estimators_[i].coef_, X_test[0]) + regr.estimators_[i].intercept_)

In [ ]:
# If you want to convert this notebook to pdf 
!jupyter nbconvert --to slides Module2.ipynb --post serve
